In [2]:
import json
import requests

# fetch ticker from the openfigi api
def fetch_ticker_from_openfigi(isin: str, exchcode: str = 'US') -> str:
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    openfigi_headers = {'Content-Type': 'application/json'}  # Changed to 'application/json'
    openfigi_data = [{"idType": "ID_ISIN", "idValue": isin, "exchCode": exchcode}]  # Formatted as JSON list
    openfigi_response = requests.post(openfigi_url, headers=openfigi_headers, json=openfigi_data)  # Used json parameter instead of data
    openfigi_response_json = openfigi_response.json()
    if openfigi_response_json:
        if 'data' in openfigi_response_json[0]:
            if 'ticker' in openfigi_response_json[0]['data'][0]:
                return openfigi_response_json[0]['data'][0]['ticker']
    return None


ticker = fetch_ticker_from_openfigi('DK0062498333')
print("Ticker:", ticker)


Ticker: NONOF


In [3]:
fetch_ticker_from_openfigi('DK0062498333')

'NONOF'

In [11]:
import json
import requests
import time

# Fetch tickers from the openfigi api for a list of ISINs
def fetch_tickers_from_openfigi(isins: list, exchcode: str = 'US') -> dict:
    
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    openfigi_headers = {'Content-Type': 'application/json'}
    

    # Prepare request data for multiple ISINs
    openfigi_data = []
    #openfigi_data = [{"idType": "ID_ISIN", "idValue": isin, "exchCode": exchcode} for isin in isins]
    for isin in isins:
        if isin == '-':
            continue
        openfigi_data.append({"idType": "ID_ISIN", "idValue": isin, "exchCode": exchcode})

    # Create a dictionary to store the tickers
    tickers_dict = {}
    count = 0
    # Process each ISIN
    for idx, data in enumerate(openfigi_data):
        # Send request
        openfigi_response = requests.post(openfigi_url, headers=openfigi_headers, json=[data])
        openfigi_response_json = openfigi_response.json()

        # Update ticker dictionary
        if 'data' in openfigi_response_json[0] and 'ticker' in openfigi_response_json[0]['data'][0]:
            tickers_dict[isins[idx]] = openfigi_response_json[0]['data'][0]['ticker']
        else:
            tickers_dict[isins[idx]] = None

        time.sleep(3)  # api limit is 25 request per minute
        count+= 1
        if count == 3:
            break

    with open('tickers_response.json', 'w') as file:
        json.dump(openfigi_response_json, file)
    
    return tickers_dict

# Example usage
#isin_list = ['DK0062498333', 'US02079K1079', 'US91324P1021']  # Add more ISINs as needed
#tickers = fetch_tickers_from_openfigi(isin_list)
# Save the response in a JSON file


In [10]:
import pandas as pd
companies_df = pd.read_excel('data/ishares-msci-world-etf-holdings-29.02.2024.xlsx')
fetch_tickers_from_openfigi(companies_df['ISIN'].tolist())


[{'data': [{'figi': 'BBG000BPH459', 'name': 'MICROSOFT CORP', 'ticker': 'MSFT', 'exchCode': 'US', 'compositeFIGI': 'BBG000BPH459', 'uniqueID': None, 'securityType': 'Common Stock', 'marketSector': 'Equity', 'shareClassFIGI': 'BBG001S5TD05', 'uniqueIDFutOpt': None, 'securityType2': 'Common Stock', 'securityDescription': 'MSFT'}]}]
[{'data': [{'figi': 'BBG000B9XRY4', 'name': 'APPLE INC', 'ticker': 'AAPL', 'exchCode': 'US', 'compositeFIGI': 'BBG000B9XRY4', 'uniqueID': None, 'securityType': 'Common Stock', 'marketSector': 'Equity', 'shareClassFIGI': 'BBG001S5N8V8', 'uniqueIDFutOpt': None, 'securityType2': 'Common Stock', 'securityDescription': 'AAPL'}]}]
[{'data': [{'figi': 'BBG000BBJQV0', 'name': 'NVIDIA CORP', 'ticker': 'NVDA', 'exchCode': 'US', 'compositeFIGI': 'BBG000BBJQV0', 'uniqueID': None, 'securityType': 'Common Stock', 'marketSector': 'Equity', 'shareClassFIGI': 'BBG001S5TZJ6', 'uniqueIDFutOpt': None, 'securityType2': 'Common Stock', 'securityDescription': 'NVDA'}]}]


{'US5949181045': 'MSFT', 'US0378331005': 'AAPL', 'US67066G1040': 'NVDA'}

In [3]:
# if ISIN = '-' 

['US5949181045',
 'US0378331005',
 'US67066G1040',
 'US0231351067',
 'US30303M1027',
 'US02079K3059',
 'US02079K1079',
 'US5324571083',
 'US11135F1012',
 'US88160R1014',
 'US46625H1005',
 'US0846707026',
 'US91324P1021',
 'US92826C8394',
 'US30231G1022',
 'US57636Q1040',
 'US4781601046',
 'DK0062498333',
 'NL0010273215',
 'US4370761029',
 'US7427181091',
 'US22160K1051',
 'US58933Y1055',
 'US0079031078',
 'US00287Y1091',
 'US79466L3024',
 'CH0038863350',
 'US1667641005',
 'US64110L1061',
 'US9311421039',
 'JP3633400001',
 'US00724F1012',
 'FR0000121014',
 'US1912161007',
 'US0605051046',
 'IE00B4BNMY34',
 'US7134481081',
 'US8835561023',
 'IE000S9YS762',
 'US5801351017',
 'CH0012005267',
 'US0028241000',
 'GB00BP6MXD84',
 'US2546871060',
 'US9497461015',
 'DE0007164600',
 'GB0009895292',
 'US17275R1023',
 'US4612021034',
 'US68389X1054',
 'CH0012032048',
 'US4581401001',
 'US2358511028',
 'US7475251036',
 'US20030N1019',
 'US1491231015',
 'US3696043013',
 'US92343V1044',
 'US4592001014